In [3]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd
import numpy as np
import ast
import json

In [4]:
import sys

def loadingBar(count,total,size):
    percent = float(count)/float(total)*100
    sys.stdout.write("\r" + str(int(percent)).rjust(3,'0')+ "% "+"/"+str("100%").rjust(3,'0') +
                     ' [' + '='*int(percent/4)*size + ' '*(10-int(percent/10))*size + ']')

## Azure 

In [5]:
web_azure = requests.get('https://azureprice.net/')
s = BeautifulSoup(web_azure.content, "html.parser")
# All Region option
option = s.find_all("option")
option = option[24:-10]
#option

In [6]:
azure_query = []
azure_r = []
azure_q = []
for i in range(len(option)):
    o = str(option[i])
    o1 = o.replace("selected=\"\"", "")
    o1 = o1.split("\"")[1]
    o2 = o.split(">")[1]
    o2 = o2.split("<")[0].split(" (")
    http = "https://api.azureprice.net/api/v1/price?currency=USD&region="+o1+"&tier=standard&paymentType=payasyougo"
    azure_q.append(o1)
    azure_r.append(o2[0])
    azure_query.append(http)
#azure_query


In [7]:
# Get data
azure_data = []
percent_value = 0
for i in range(len(azure_query)):
    web = requests.get(azure_query[i])
    s = BeautifulSoup(web.content, "html.parser")
    js = json.loads(s.text)
    d = js["listVmInformations"]
    df = pd.DataFrame(d)
    df = df[["name","numberOfCores","memoryInMB","linuxPrice","windowsPrice","gpUs","regionName"]]
    azure_data.append(df)
    percent_value += 1
    loadingBar(percent_value,len(azure_query),2)

100% /100% [==================================================]

In [8]:
azure_data = pd.concat(azure_data)
azure_data["vendor"] = "azure"
azure_data = azure_data.rename(columns={"name":"vm name", "numberOfCores":"vCPUs",
                                       "memoryInMB":"memory", "linuxPrice":"Linux cost",
                                       "windowsPrice":"Window cost","gpUs":"GPUs",
                                        "regionName":"Region"})
azure_data["Region"].unique()

array(['Central US', 'East US', 'East US 2', 'North Central US',
       'South Central US', 'West Central US', 'West US', 'West US 2',
       'West US 3', 'US Gov AZ', 'US Gov TX', 'US Gov Virginia',
       'Canada Central', 'Canada East', 'France Central', 'France South',
       'Germany North', 'Germany West Central', 'North Europe',
       'Norway East', 'Norway West', 'Sweden Central',
       'Switzerland North', 'Switzerland West', 'UK South', 'UK West',
       'West Europe', 'Australia Central', 'Australia Central2',
       'Australia East', 'Australia Southeast', 'Central India',
       'East Asia', 'Japan East', 'Japan West', 'Korea Central',
       'Korea South', 'Southeast Asia', 'South India', 'West India',
       'Jio India West', 'UAE Central', 'UAE North', 'South Africa North',
       'South Africa West', 'Brazil South', 'Brazil Southeast'],
      dtype=object)

### test rec

In [9]:
a = azure_data[azure_data["Region"]=="Central US"]
a

,vm name,vCPUs,memory,Linux cost,Window cost,GPUs,Region,vendor
0,Basic_A0,1,768,0.018,0.018,0,Central US,azure
1,Basic_A1,1,1792,0.025,0.035,0,Central US,azure
2,Basic_A2,2,3584,0.085,0.139,0,Central US,azure
3,Basic_A3,4,7168,0.188,0.308,0,Central US,azure
4,Basic_A4,8,14336,0.376,0.616,0,Central US,azure
...,...,...,...,...,...,...,...,...
551,Standard_NC24s_v3,24,458752,13.831,14.935,4,Central US,azure
552,Standard_NC4as_T4_v3,4,28672,0.594,0.778,1,Central US,azure
553,Standard_NC64as_T4_v3,64,450560,4.907,7.851,4,Central US,azure
554,Standard_NC6s_v3,6,114688,3.458,3.734,1,Central US,azure


In [10]:
select = np.random.randint(0,30) # assume selected VM
#select=11
region = a.iloc[select,6]
cpu = a.iloc[select,1]
memory = a.iloc[select,2]
gpu = a.iloc[select,5]
name = a.iloc[select,0]
vendor = a.iloc[select,7]
operation = "window"

print(a.iloc[select])

vm name        Standard_A2m_v2
vCPUs                        2
memory                   16384
Linux cost               0.129
Window cost              0.195
GPUs                         0
Region              Central US
vendor                   azure
Name: 10, dtype: object


In [24]:
def rec_azure(vendor,region,cpu,memory,gpu,name,operation):
    cost=0
    text = ""
    for i in range(len(azure_q)):
        if region==azure_r[i]:
            region_q = azure_q[i]
    http = "https://api.azureprice.net/api/v1/price?currency=USD&region="+region_q+"&tier=standard&paymentType=payasyougo"
    #print(http)
    web = requests.get(http)
    s = BeautifulSoup(web.content, "html.parser")
    js = json.loads(s.text)
    d = js["listVmInformations"]
    df = pd.DataFrame(d)
    df = df[["name","numberOfCores","memoryInMB","linuxPrice","windowsPrice","gpUs","regionName"]]
    df = df.rename(columns={"name":"vm name", "numberOfCores":"vCPUs",
                                       "memoryInMB":"memory", "linuxPrice":"Linux cost",
                                       "windowsPrice":"Window cost","gpUs":"GPUs",
                                        "regionName":"Region"})
    #print(df)
    #print(df["Window cost"].std())
    select = df[df["vm name"]==name]
    print(select)
    # sep window & linux
    if operation == "window":
        cost = select.iloc[0,4]
        text= "Window cost"
    elif operation == "linux":
        cost = select.iloc[0,3]
        text = "Linux cost"
        
    
    up_cost = cost+cost*df[text].std()
    down_cost = cost-0.6*cost
    
        # find range of CPU
    cpu = select.iloc[0,1]
    cpu_con_down = 0.5*float(cpu)
    cpu_con_up = 2.5*float(cpu)
        # find range of memory
    memory = select.iloc[0,2]
    mem_con_down = 0.5*float(memory)
    mem_con_up = 2*float(memory)
    
    #if gpu=0:
    if gpu==0:
        rec = df[df[text]!=0]
        rec = rec[(rec[text]>=down_cost) & (rec[text]<=up_cost)].reset_index().drop("index", axis=1)
        rec = rec[(rec['vCPUs']>=cpu_con_down) & (rec['vCPUs']<=cpu_con_up)].reset_index().drop("index", axis=1)
        rec = rec[(rec['memory']>=mem_con_down) & (rec['memory']<=mem_con_up)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by=text).reset_index().drop("index", axis=1)
        #print(rec)
        for i in range(len(rec)):
                rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
                rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
        #print(rec)
        rec = rec[(rec["saving"]>=-100)&(rec["saving"]<=100)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)
        
        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.iloc[i,7]/rec["point"].std())+(rec.iloc[i,8]/rec["saving"].std())
        rec = rec.sort_values(by = "kpi", ascending=False).reset_index().drop("index", axis=1)
        #print(rec)
    #rec = rec[rec["vm name"]!=name]
    #print(rec)
    
    elif gpu!=0:
        rec = df[df["GPUs"]!=0].reset_index().drop("index", axis=1)
        print(type(rec))
        for i in range(len(rec)):
            rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
            rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
    
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)

        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.iloc[i,7]/rec["point"].std())+((rec.iloc[i,8]/rec["saving"].std()))
        rec = rec.sort_values(by = "kpi", ascending=True).reset_index().drop("index", axis=1)
    rec = rec[rec["vm name"]!=name] 
    #print(rec)
    #a = rec.to_json()
    #parsed = json.loads(a)
    #print(a)
    return rec



In [25]:
a = rec_azure("azure","East US 2",4,16384,0,"Standard_B4ms","window") # Customer selected
a

          vm name  vCPUs  memory  Linux cost  Window cost  GPUs     Region
28  Standard_B4ms      4   16384       0.166        0.182     0  East US 2


,vm name,vCPUs,memory,Linux cost,Window cost,GPUs,Region,point,saving,kpi
1,Standard_A2m_v2,2,16384,0.1190,0.1800,0,East US 2,-0.500,1.098901,-1.730111
2,Standard_B2ms,2,8192,0.0832,0.0912,0,East US 2,-1.000,49.890110,-1.780059
3,Standard_E2ps_v5,2,16384,0.1010,0.1930,0,East US 2,-0.500,-6.043956,-1.981749
4,Standard_E2as_v5,2,16384,0.1130,0.2050,0,East US 2,-0.500,-12.637363,-2.214030
5,Standard_E2pds_v5,2,16384,0.1150,0.2070,0,East US 2,-0.500,-13.736264,-2.252743
6,Standard_E2_v4,2,16384,0.1260,0.2180,0,East US 2,-0.500,-19.780220,-2.465667
7,Standard_E2s_v5,2,16384,0.1260,0.2180,0,East US 2,-0.500,-19.780220,-2.465667
8,Standard_E2_v5,2,16384,0.1260,0.2180,0,East US 2,-0.500,-19.780220,-2.465667
9,Standard_E2s_v4,2,16384,0.1260,0.2180,0,East US 2,-0.500,-19.780220,-2.465667
10,Standard_E2a_v4,2,16384,0.1260,0.2180,0,East US 2,-0.500,-19.780220,-2.465667


In [21]:
a.iloc[0:15]

,vm name,vCPUs,memory,Linux cost,Window cost,GPUs,Region,point,saving,kpi
1,Standard_A2m_v2,2,16384,0.1190,0.1800,0,East US 2,-0.5,1.098901,-1.730111
2,Standard_B2ms,2,8192,0.0832,0.0912,0,East US 2,-1.0,49.890110,-1.780059
3,Standard_E2ps_v5,2,16384,0.1010,0.1930,0,East US 2,-0.5,-6.043956,-1.981749
4,Standard_E2as_v5,2,16384,0.1130,0.2050,0,East US 2,-0.5,-12.637363,-2.214030
5,Standard_E2pds_v5,2,16384,0.1150,0.2070,0,East US 2,-0.5,-13.736264,-2.252743
6,Standard_E2_v4,2,16384,0.1260,0.2180,0,East US 2,-0.5,-19.780220,-2.465667
7,Standard_E2s_v5,2,16384,0.1260,0.2180,0,East US 2,-0.5,-19.780220,-2.465667
8,Standard_E2_v5,2,16384,0.1260,0.2180,0,East US 2,-0.5,-19.780220,-2.465667
9,Standard_E2s_v4,2,16384,0.1260,0.2180,0,East US 2,-0.5,-19.780220,-2.465667
10,Standard_E2a_v4,2,16384,0.1260,0.2180,0,East US 2,-0.5,-19.780220,-2.465667


In [406]:
#rec_azure(vendor,region,cpu,memory,gpu,name,operation)

## AWS

In [372]:
aws_gpu = ['g4dn.12xlarge', 'g4dn.16xlarge', 'g4dn.2xlarge', 'g4dn.4xlarge',
       'g4dn.8xlarge', 'g4dn.metal', 'g4dn.xlarge', 'g5.12xlarge',
       'g5.16xlarge', 'g5.24xlarge', 'g5.2xlarge', 'g5.48xlarge',
       'g5.4xlarge', 'g5.8xlarge', 'g5.xlarge', 'p2.16xlarge',
       'p2.8xlarge', 'p2.xlarge', 'g2.2xlarge', 'g2.8xlarge',
       'g3.16xlarge', 'g3.4xlarge', 'g3.8xlarge', 'g3s.xlarge',
       'g5g.16xlarge', 'g5g.2xlarge', 'g5g.4xlarge', 'g5g.8xlarge',
       'g5g.metal', 'g5g.xlarge', 'p3.16xlarge', 'p3.2xlarge',
       'p3.8xlarge', 'p4d.24xlarge', 'g4ad.16xlarge', 'g4ad.2xlarge',
       'g4ad.4xlarge', 'g4ad.8xlarge', 'g4ad.xlarge', 'p3dn.24xlarge',
       'p4de.24xlarge']
n_gpu = [4, 1, 1, 1, 1, 8, 1, 4, 1, 4, 1, 8, 1, 1, 1, 16, 8, 1, 1, 4, 4, 1, 2, 
         1, 2, 1, 1, 1, 2, 1, 8, 1, 4, 8, 4, 1, 1, 2, 1, 8, 8]

In [373]:
#web_aws = requests.get('https://instances.vantage.sh')
#s = BeautifulSoup(web_aws.content, "html.parser")
#option = s.find_all("a", attrs={"class":"dropdown-item"})
#option = option[0:25]
#option

In [374]:
#aws_query = []
#for i in range(len(option)):
    #o = str(option[i]).split(">")[1].split("<")[0]
    #o = o.replace("-", " ")
    #o = o.replace("Europe","EU")
    #aws_query.append(o)
#aws_query

In [407]:
def rec_aws(vendor,region,cpu,memory,gpu,name,operation):
    reg = region.replace(" ", "%20")
    if operation == "linux":
        http = "https://b0.p.awsstatic.com/pricing/2.0/meteredUnitMaps/ec2/USD/current/ec2-ondemand-without-sec-sel/"+str(reg)+"/Linux/index.json?"
    else:
        http = "https://b0.p.awsstatic.com/pricing/2.0/meteredUnitMaps/ec2/USD/current/ec2-ondemand-without-sec-sel/"+str(reg)+"/Windows/index.json?"
    
    #print(http)
    web =requests.get(http)
    s = BeautifulSoup(web.content, "html.parser")
    js = json.loads(s.text)
    d = js["regions"][region]
    
    df = pd.DataFrame.from_dict(d, orient='index')
    df = df[["Instance Type","vCPU","Memory","price"]].reset_index().drop("index", axis=1)
    df = df.rename(columns = {"Instance Type":"vm name", "vCPU":"vCPUs","Memory":"memory"})

    #print(df[df["vm name"]==name])
    for i in range(len(df)):
        df.iloc[i,1] = float(df.iloc[i,1])
        df.iloc[i,2] = float(df.iloc[i,2].replace(" GiB", ""))
        df.iloc[i,3] = float(df.iloc[i,3])
        for j in range(len(aws_gpu)):
            if df.iloc[i,0]==aws_gpu[j]:
                df.loc[i,"GPUs"] = n_gpu[j]
    df["GPUs"] = df["GPUs"].fillna(0)
    df["region"] = region
    df["vendor"] = "aws"

    print(df[df["vm name"]==name])
    cost = df[df["vm name"]==name].iloc[0,3]
    text = "price"
    
    up_cost = cost+cost*df[text].std()
    down_cost = cost-0.6*cost
        # find range of CPU
    cpu_con_down = 0.5*float(cpu)
    cpu_con_up = 2.5*float(cpu)
        # find range of memory
    mem_con_down = 0.5*float(memory)
    mem_con_up = 2*float(memory)
    
    #if gpu=0:
    if gpu==0:
        rec = df[df[text]!=0]
        rec = rec[(rec[text]>=down_cost) & (rec[text]<=up_cost)].reset_index().drop("index", axis=1)
        rec = rec[(rec['vCPUs']>=cpu_con_down) & (rec['vCPUs']<=cpu_con_up)].reset_index().drop("index", axis=1)
        rec = rec[(rec['memory']>=mem_con_down) & (rec['memory']<=mem_con_up)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by=text).reset_index().drop("index", axis=1)
        #print(rec)
        for i in range(len(rec)):
                rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
                rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
        #print(rec)
        rec = rec[(rec["saving"]>=-100)&(rec["saving"]<=100)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)
        
        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.loc[i,"point"]/rec["point"].std())+(rec.loc[i,"saving"]/rec["saving"].std())
        rec = rec.sort_values(by = "kpi", ascending=False).reset_index().drop("index", axis=1)
        #print(rec)
   
    
    elif gpu!=0:
        rec = df[df["GPUs"]!=0].reset_index().drop("index", axis=1)
        print(type(rec))
        for i in range(len(rec)):
            rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
            rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
    
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)
    
        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.loc[i,"point"]/rec["point"].std())+((rec.loc[i,"saving"]/rec["saving"].std()))
        rec = rec.sort_values(by = "kpi", ascending=True).reset_index().drop("index", axis=1)
        
        
    rec = rec[rec["vm name"]!=name] 
    #print(rec)
        

    return rec
    
    

In [408]:
rec_aws("aws","Asia Pacific (Singapore)",2,16,0,"r5dn.large","window") # Customer selected


        vm name vCPUs memory  price  GPUs                    region vendor
133  r5dn.large   2.0   16.0  0.292   0.0  Asia Pacific (Singapore)    aws


,vm name,vCPUs,memory,price,GPUs,region,vendor,point,saving,kpi
0,t3a.xlarge,4.0,16.0,0.2624,0.0,Asia Pacific (Singapore),aws,1.000000,10.136986,1.368625
1,t2.xlarge,4.0,16.0,0.2746,0.0,Asia Pacific (Singapore),aws,1.000000,5.958904,1.269868
2,t3.xlarge,4.0,16.0,0.2848,0.0,Asia Pacific (Singapore),aws,1.000000,2.465753,1.187300
3,r5a.xlarge,4.0,32.0,0.456,0.0,Asia Pacific (Singapore),aws,2.000000,-56.164384,0.930477
4,t3a.large,2.0,8.0,0.122,0.0,Asia Pacific (Singapore),aws,-0.500000,58.219178,0.811618
5,t3.large,2.0,8.0,0.1332,0.0,Asia Pacific (Singapore),aws,-0.500000,54.383562,0.720956
6,r6i.xlarge,4.0,32.0,0.488,0.0,Asia Pacific (Singapore),aws,2.000000,-67.123288,0.671441
7,r5.xlarge,4.0,32.0,0.488,0.0,Asia Pacific (Singapore),aws,2.000000,-67.123288,0.671441
8,t2.large,2.0,8.0,0.1448,0.0,Asia Pacific (Singapore),aws,-0.500000,50.410959,0.627055
9,r5ad.xlarge,4.0,32.0,0.502,0.0,Asia Pacific (Singapore),aws,2.000000,-71.917808,0.558113


## GCP

In [409]:
def rec_gcp(vendor,region,cpu,memory,gpu,name,operation):
    cost=0
    text = operation
    web = requests.get("https://gcpinstances.doit-intl.com/data.json")
    s = BeautifulSoup(web.content, "html.parser")
    js = json.loads(s.text)
    vm_type = list(js.keys())
    vm_type = vm_type[2:-1]
    specific = []
    for i in range(len(vm_type)):
        s = list(js[vm_type[i]].keys())
        specific.append(s)
    detail = list(js[vm_type[0]][specific[0][0]].keys())
    gcp = []
    for i in range(len(vm_type)):
        for j in range(len(specific[i])):
            df = pd.DataFrame.from_dict(js[vm_type[i]][specific[i][j]][detail[0]]).transpose().reset_index()
            df = df.rename(columns = {"ondemand":"Linux cost", "index":"Region"})
            df["vCPUs"] = js[vm_type[i]][specific[i][j]][detail[1]]["cores"]
            df["memory"] = js[vm_type[i]][specific[i][j]][detail[1]]["memory"]

            df["GPUs"] = js[vm_type[i]][specific[i][j]][detail[1]]["gpu"]
            df["vm name"] = specific[i][j]
            df["Window cost"] = df["Linux cost"] + 0.046*df["vCPUs"]
            c = []
            for k in range(len(df)):
                check = re.search('\d',df.iloc[k,0])
                check = str(check)
                if check=='None':
                    c.append(k)
            df = df.drop(c, axis=0).reset_index().drop("index", axis=1)
            df = df[["vm name","vCPUs","memory","Linux cost", "Window cost","GPUs", "Region"]]
            gcp.append(df)
    gcp_data = pd.concat(gcp)
    gcp_data = gcp_data.fillna(0)
    gcp_data = gcp_data[gcp_data["Region"]==region]
    #gcp_data
    
    select = gcp_data[gcp_data["vm name"]==name]
    print(select)
    # sep window & linux
    if operation == "window":
        cost = select.iloc[0,4]
        text= "Window cost"
    elif operation == "linux":
        cost = select.iloc[0,3]
        text = "Linux cost"
        
    up_cost = cost+cost*gcp_data[text].std()
    down_cost = cost-0.6*cost
        # find range of CPU
    cpu_con_down = 0.5*float(cpu)
    cpu_con_up = 2.5*float(cpu)
        # find range of memory
    mem_con_down = 0.5*float(memory)
    mem_con_up = 2*float(memory)
    
    #if gpu=0:
    if gpu==0:
        rec = gcp_data[gcp_data[text]!=0]
        rec = rec[(rec[text]>=down_cost) & (rec[text]<=up_cost)].reset_index().drop("index", axis=1)
        rec = rec[(rec['vCPUs']>=cpu_con_down) & (rec['vCPUs']<=cpu_con_up)].reset_index().drop("index", axis=1)
        rec = rec[(rec['memory']>=mem_con_down) & (rec['memory']<=mem_con_up)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by=text).reset_index().drop("index", axis=1)
        #print(rec)
        for i in range(len(rec)):
                rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
                rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
        #print(rec)
        rec = rec[(rec["saving"]>=-100)&(rec["saving"]<=100)].reset_index().drop("index", axis=1)
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)
        
        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.iloc[i,7]/rec["point"].std())+(rec.iloc[i,8]/rec["saving"].std())
        rec = rec.sort_values(by = "kpi", ascending=False).reset_index().drop("index", axis=1)
        #print(rec)
    #rec = rec[rec["vm name"]!=name]
    #print(rec)
    
    elif gpu!=0:
        rec = df[df["GPUs"]!=0].reset_index().drop("index", axis=1)
        print(type(rec))
        for i in range(len(rec)):
            rec.loc[i,"point"] = (rec.iloc[i,1]-cpu)/cpu + (rec.iloc[i,2]-memory)/memory
            rec.loc[i,"saving"] = (1-rec.loc[i,text]/cost)*100
    
        rec = rec.sort_values(by = ["point","saving"], ascending=False).reset_index().drop("index", axis=1)
    
        for i in range(len(rec)):
            rec.loc[i,"kpi"] = (rec.iloc[i,7]/rec["point"].std())+((rec.iloc[i,8]/rec["saving"].std()))
        rec = rec.sort_values(by = "kpi", ascending=True).reset_index().drop("index", axis=1)
    rec = rec[rec["vm name"]!=name] 
    #print(rec)
    #a = rec.to_json()
    #parsed = json.loads(a)
    #print(a)
    return rec


In [410]:
rec_gcp("gcp","us-central1",48,192,0,"t2d-standard-48","linux") # Customer selected

           vm name  vCPUs  memory  Linux cost  Window cost  GPUs       Region
0  t2d-standard-48     48   192.0     2.02781      4.23581     0  us-central1


,vm name,vCPUs,memory,Linux cost,Window cost,GPUs,Region,point,saving,kpi
0,n2d-highmem-48,48,384.0,2.73552,4.94352,0,us-central1,1.000000,-34.900213,0.526097
1,n2d-standard-96,96,384.0,4.05562,8.47162,0,us-central1,2.000000,-100.000000,0.299421
2,n2d-highmem-32,32,256.0,1.82368,3.29568,0,us-central1,0.000000,10.066525,0.250924
3,e2-standard-32,32,128.0,1.07218,2.54418,0,us-central1,-0.666667,47.126210,0.244003
4,t2a-standard-48,48,192.0,1.84800,4.05600,0,us-central1,0.000000,8.867202,0.221029
5,n2d-standard-80,80,320.0,3.37968,7.05968,0,us-central1,1.333333,-66.666502,0.199618
6,n2d-standard-64,64,256.0,2.70374,5.64774,0,us-central1,0.666667,-33.333005,0.099815
7,c2d-highmem-32,32,256.0,1.95952,3.43152,0,us-central1,0.000000,3.367673,0.083945
8,t2d-standard-60,60,240.0,2.53476,5.29476,0,us-central1,0.500000,-24.999877,0.074858
9,t2a-standard-32,32,128.0,1.23200,2.70400,0,us-central1,-0.666667,39.244801,0.047546
